### Dropbox link for beans20190718-trim.nwb (2 GB): https://drive.google.com/file/d/1-0xdEn8oVT6lexBc8mlv7ls84oUvxU4u/view?usp=sharing

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os
from pathlib import Path

# CHANGE ME TO THE BASE DIRECTORY FOR DATA STORAGE ON YOUR SYSTEM
# data_dir = Path('/Users/loren/data/nwb_builder_test_data') 
data_dir = Path('/mnt/c/Users/Ryan/Documents/NWB_Data/Frank Lab Data/')
# data_dir = Path('/stelmo/nwb/')

os.environ['NWB_DATAJOINT_BASE_DIR'] = str(data_dir)
os.environ['KACHERY_STORAGE_DIR'] = str(data_dir / 'kachery-storage')
os.environ['DJ_SUPPORT_FILEPATH_MANAGEMENT'] = 'TRUE'

raw_dir = data_dir / 'raw'
analysis_dir = data_dir / 'analysis'

### Before running the rest of the notebook, make sure the following exist:
- data_dir
- a 'raw' subdirectory in data_dir (e.g. data_dir/raw)
  - place 'beans20190718-trim.nwb' in data_dir/raw
- an 'analysis' subdirectory in data_dir (e.g. data_dir/analysis)
- a kachery-storage subdirectory in data_dir (e.g. data_dir/kachery-storage)

In [ ]:
import datajoint as dj
dj.config['database.host'] = 'localhost'  # CHANGE THESE AS NEEDED
dj.config['database.user'] = 'root'
dj.config['database.password'] = 'tutorial'
dj.config['stores'] = {
  'raw': {
    'protocol': 'file',
    'location': str(raw_dir),
    'stage' : str(raw_dir)
  },
  'analysis': {
    'protocol': 'file',
    'location': str(analysis_dir),
    'stage': str(analysis_dir)
  }
}

import nwb_datajoint as nd

import warnings
warnings.simplefilter('ignore')

In [ ]:
nd.insert_sessions(['beans20190718-trim.nwb'])

#### Let's look at the core schema (note that schema = database_table). 
First, Nwbfile:

In [ ]:
nd.common.Nwbfile()

Each NWB file defines a session which also contains information about the subject, institution, etc.:

In [ ]:
nd.common.Session()

We can use the datajoint `Diagram` method to represent the relationship between Nwbfile and Session: 
A Session is identified by a Nwbfile (the primary key; indicated above with the black header text 'nwb_file_name')

In [ ]:
dj.Diagram(nd.common.Nwbfile()) + dj.Diagram(nd.common.Session())

The solid line indicates that the Nwbfile is the primary key for Session, so each Session has exactly one Nwbfile associated with it.

The session also contains references to other schema, including Subject, Institution, etc. 

In [ ]:
dj.Diagram(nd.common.Nwbfile()) + dj.Diagram(nd.common.Session()) + dj.Diagram(nd.common.Subject()) + dj.Diagram(nd.common.Institution()) + dj.Diagram(nd.common.Lab())

Now let's look at the raw data schema, which contains a reference to the continuous electrophysiology data at the full 20 KHz sampling rate:

In [ ]:
nd.common.Raw()

The raw data has two primary keys: the Session and an IntervalList that defines the set of time intervals for which the data are valid:

In [ ]:
dj.Diagram(nd.common.Subject()) + dj.Diagram(nd.common.Session()) + dj.Diagram(nd.common.Nwbfile()) + dj.Diagram(nd.common.Raw())

The raw data also has a reference to an NWB object id, which allows direct loading of an NWB object from the file. We can get this raw data object with a simple DataJoint query:

In [ ]:
raw_data = (nd.common.Raw() & {'nwb_file_name' : 'beans20190718-trim_.nwb'}).fetch_nwb()[0]
raw_data

Thus, `raw_data['raw']` is the electrical series from the NWB file from which data can be retrieved and analyzed.

A couple of notes here:

1) `nd.common.Raw() & {'nwb_file_name' : 'beans20190718-trim_.nwb'}` matches elements of the schema with a dictionary with 'nwb_file_name' as the key and, in this case, 'beans20190718-trim_.nwb' as the value. The file name can also be retrieved from the first row of the Nwbfile table: `nwb_file_name = nd.common.Nwbfile().fetch1()['nwb_file_name']`

2) `fetch_nwb()` is a special function that was added to the Raw schema to return NWB objects. 


### The first analysis one might do on raw data is to extract the local field potential from a set of electrodes, and there are LFP-related schema for that:

In [ ]:
dj.Diagram(nd.common.Session()) + dj.Diagram(nd.common.Nwbfile()) + dj.Diagram(nd.common.Raw()) + dj.Diagram(nd.common.LFP())

Note here that the LFP schema has an entry from the AnalysisNwbfile schema. The AnalysisNwbfile is created when the LFP is created and serves to store the LFP time series. It also contains the key metadata from the parent Nwbfile, so it's easy to know which subject, etc. it came from.

Finally, here's a full diagram of all the schema:

In [ ]:
dj.Diagram(nd.common.Session())-2+5

The "kachery" entries refer to a file sharing system developed by Jeremy Magland and colleagues, but you do not need to use that to use this repo.